In [49]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

In [52]:
!pwd

/home/timjab92/code/timjab92/cryptotrading-indicator/notebooks


In [53]:
model = load_model("../model.joblib")

In [54]:
import requests
def get_coingecko():
    #ohlc
    url = "https://api.coingecko.com/api/v3/coins/bitcoin/ohlc?vs_currency=usd&days=30"
    response = requests.get(url).json()
    #cleaning
    data_api = pd.DataFrame(response, columns = ['unix_time','open', 'high', 'low', 'close'])
    data_api["Date"] = pd.to_datetime(data_api["unix_time"], unit='ms')
    data_api = data_api.drop(columns='unix_time').set_index('Date')

    #volume
    url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=30"
    response = requests.get(url).json()

    volume = pd.DataFrame(response['total_volumes'], columns=["unix_time","volume"])
    volume['date'] = pd.to_datetime(pd.to_datetime(volume['unix_time'],unit='ms').dt.strftime("%Y/%m/%d, %H:00:00"))
    volume = volume.drop(columns='unix_time').set_index('date')

    #resample hourly into 4h
    volume = volume.resample("4H").mean()

    #concatinate
    volume = volume[-180:]
    data_api = data_api[-181:-1]
    full = pd.concat([data_api, volume], axis=1)
    full.columns=['open', 'high', 'low', 'close', 'volume']

    for x in ['open', 'high', 'low', 'close']:
        full[f'log_{x}'] = full[x].apply(lambda x: np.log(x))

    data_api = full.copy()

    return data_api


def feature_engineer(data):
    """
    Adds the EMAs, StochRSI, BollingerBands and Volume Rate of Change to the dataframe
    """
    add_ema(data)
    add_stoch_rsi(data)
    add_bollinger(data,data.log_close)
    add_vol_roc(data)
    return data

## add other features

def add_ema(data, tspan=[12,26,20,50,34,55]):
    """
    Adds Exponential Moving Averages (EMA) to the dataframe. The default timeframes are 12,26,20,50,34 and 55.
    """
    for t in tspan:
        data[f'ema{t}'] = data.log_close.ewm(span=t).mean()
    return data


def computeRSI(data, window=14):
    """
    Computes the Relative Stregth Index for a given dataset and the window can be defined. Its default value is 14.
    """
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=window-1 so we get decay alpha=1/window
    up_chg_avg   = up_chg.ewm(com=window-1 , min_periods=window).mean()
    down_chg_avg = down_chg.ewm(com=window-1 , min_periods=window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi


def stoch_rsi(rsi, d_window=3, k_window=3, window=14):
    """
    Computes the stochastic RSI. Default values are d=3, k=3, window=14.
    """
    minrsi = rsi.rolling(window=window, center=False).min()
    maxrsi = rsi.rolling(window=window, center=False).max()
    stoch = ((rsi - minrsi) / (maxrsi - minrsi)) * 100
    K = stoch.rolling(window=k_window, center=False).mean()
    D = K.rolling(window=d_window, center=False).mean() 
    return K, D  


def add_stoch_rsi(data, d_window=3, k_window=3, window=14):
    data['rsi'] = computeRSI(data['log_close'], window)
    data['K'], data['D'] = stoch_rsi(data['rsi'], d_window, k_window, window)
    return data


def get_bollinger_bands(prices, rate=20):
    sma = prices.rolling(rate).mean() # <-- Get SMA for 20 days
    std = prices.rolling(rate).std() # <-- Get rolling standard deviation for 20 days
    bollinger_up = sma + std * 2 # Calculate top band
    bollinger_down = sma - std * 2 # Calculate bottom band
    return sma, bollinger_up, bollinger_down


def add_bollinger(data, prices, rate=20):
    data['sma'], data['bollinger_up'], data['bollinger_down'] = get_bollinger_bands(prices)
    return data


def add_vol_roc(data):
    data['vol_roc'] = data.volume.pct_change()
    return data


In [55]:
def get_train_data():
    """
    Returns the raw training dataset for the price of bitcoin since 31.12.2011.
    The index is set to the date.
    """
    data = pd.read_csv("../data/BTCUSD_4hours.csv")
    data['date'] = pd.to_datetime(data.date)
    data_train = data.drop(columns="Unnamed: 0").set_index("date")
    return data_train


def minmaxscaling(data_train):
    """
    applies the minmaxscaler to the training set. Attention! Output needs to be
    defined for data_train_scaled, min1 and range1!!
    """
    minmax_scaler = MinMaxScaler(feature_range=(0, 1))
    minmax_scaler.fit(data_train)
    data_train_scaled = minmax_scaler.transform(data_train)
    #    min1 = minmax_scaler.data_min_  # [5:9] for log_prices
    #    range1 = minmax_scaler.data_range_  #[5:9]
    return data_train_scaled, minmax_scaler


def get_xgecko(length=60, horizon=1):
    """
    Calls the coingecko API and returns the data used for prediction.
    x_gecko.shape == (no_sequ , length, no_features)
    """
    x_gecko = feature_engineer(get_coingecko())[['log_close']][-length:]
    #get scaler the long way
    data_train = feature_engineer(get_train_data())[['log_close']]
    data_train_scaled, scaler = minmaxscaling(data_train)

    x_gecko_scaled = scaler.transform(x_gecko)
    x_gecko = np.array(x_gecko_scaled)
    x_gecko = np.reshape(x_gecko, (horizon, length, 1))
    return x_gecko

In [81]:
6*4

24

In [85]:
get_train_data().tail(154)

,open,high,low,close,volume,log_open,log_high,log_low,log_close
date,,,,,,,,,
2021-07-28 16:00:00,39920.657417,39958.128792,39884.646083,39923.986500,2.863361,10.594590,10.595528,10.593685,10.594672
2021-07-28 20:00:00,39985.135208,40013.786333,39953.212792,39984.792750,1.402206,10.596256,10.596973,10.595457,10.596248
2021-07-29 00:00:00,39751.033125,39771.916417,39726.004083,39748.772542,0.884538,10.590383,10.590908,10.589753,10.590326
2021-07-29 04:00:00,40003.717708,40021.152667,39986.370208,40005.281208,1.491767,10.596723,10.597158,10.596289,10.596762
2021-07-29 08:00:00,40116.683167,40139.699625,40092.638500,40117.184417,2.164342,10.599534,10.600107,10.598934,10.599546
...,...,...,...,...,...,...,...,...,...
2021-08-22 12:00:00,48769.308417,48787.711375,48749.417500,48769.737750,1.697971,10.794844,10.795222,10.794435,10.794853
2021-08-22 16:00:00,48591.665708,48605.941333,48574.903375,48591.580833,0.463362,10.791204,10.791498,10.790859,10.791202
2021-08-22 20:00:00,48807.713542,48825.874417,48791.558000,48810.168750,0.605696,10.795610,10.795982,10.795279,10.795660


In [86]:
get_coingecko().tail(60)

,open,high,low,close,volume,log_open,log_high,log_low,log_close
2021-08-17 16:00:00,46970.34,46970.34,46008.18,46008.18,3.289771e+10,10.757272,10.757272,10.736574,10.736574
2021-08-17 20:00:00,45727.59,45890.78,45710.32,45710.32,3.403946e+10,10.730457,10.734020,10.730079,10.730079
2021-08-18 00:00:00,45427.44,45427.44,44894.87,44937.31,3.509723e+10,10.723872,10.723872,10.712079,10.713024
2021-08-18 04:00:00,44534.45,45033.48,44534.45,44953.67,3.681299e+10,10.704018,10.715161,10.704018,10.713388
2021-08-18 08:00:00,45030.10,45456.91,45011.99,45456.91,3.697893e+10,10.715086,10.724520,10.714684,10.724520
2021-08-18 12:00:00,45046.34,45477.62,45046.34,45394.46,3.727284e+10,10.715447,10.724976,10.715447,10.723145
2021-08-18 16:00:00,44809.22,45339.30,44809.22,45339.30,3.694369e+10,10.710169,10.721929,10.710169,10.721929
2021-08-18 20:00:00,45753.34,45894.98,45517.30,45517.30,3.522890e+10,10.731020,10.734111,10.725848,10.725848
2021-08-19 00:00:00,44894.63,44894.63,44631.41,44796.74,3.403802e+10,10.712073,10.712073,10.706193,10.709891
2021-08-19 04:00:00,44790.95,45240.42,44678.53,44678.53,3.324865e+10,10.709761,10.719746,10.707248,10.707248


In [57]:
x_gecko = get_xgecko()

In [58]:
x_gecko.shape

(1, 60, 1)

In [59]:
data_train_scaled, scaler = minmaxscaling(feature_engineer(get_train_data())[['log_close']])

In [62]:
pred = model.predict(x_gecko)

In [64]:
pred

array([[0.9688844]], dtype=float32)

In [65]:
pred = np.exp(scaler.inverse_transform(pred))

In [72]:
pred[0][0]

47679.004